# TOPIC : COVID-19 Detection Using Chest X-ray

Members :
* Abhishek Kumar (202IT001)
* Nitin Sharma (202IT017)

## Importing relevant libraries

In [ ]:
%%capture
import numpy as np
import tensorflow as tf
import keras.backend as K
import scipy.misc
import pydot
import pandas as pd
import os
import shutil
import cv2 

from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model, load_model, Sequential
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
from keras.utils.vis_utils import model_to_dot
# from keras.utils import plot_model
from keras.initializers import glorot_uniform

from random import shuffle 
from tqdm import tqdm 
from numpy import load
from IPython.display import SVG
from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt
from keras.optimizers import Adam

%matplotlib inline
K.set_image_data_format('channels_last')
K.set_learning_phase(1)


## Dataset Creation

In [ ]:
# df = pd.read_csv('/content/metadata.csv')
# df[:2]

In [ ]:
# from_path = '/content/drive/MyDrive/ML/cv/covid-chestxray-dataset-master/images/'
# to_path = '/content/drive/MyDrive/ML/cv/dataset2/covid/'

In [ ]:
# count = 0
# files = []
# for i, rec in df.iterrows() :
#   if rec['finding'] == 'Pneumonia/Viral/COVID-19' and rec['view'] == 'PA':
#     file_name = rec['filename']
#     count = count+1
#     shutil.copy2((from_path+file_name),(to_path+file_name))
#     print("Moving Image : ", file_name)

# print("Total copied : ", count)

## Basic Pre-processing

ref : https://towardsdatascience.com/step-by-step-vgg16-implementation-in-keras-for-beginners-a833c686ae6c

In [ ]:
%%capture
!wget http://cb.lk/covid_19
!unzip covid_19

In [ ]:
width, height = 224, 224
# covid_path = "/content/drive/MyDrive/ML/cv_try/covid/"
# normal_path = "/content/drive/MyDrive/ML/cv/normal/"
img_size = (width, height, 3)

## Test and Train data generator

In [ ]:
train_datagen = image.ImageDataGenerator(
 rescale=1./255,
 shear_range=0.2,
 zoom_range=0.2,
 horizontal_flip = True)

val_dataset = image.ImageDataGenerator(
 rescale=1./255,
 shear_range=0.2,
 zoom_range=0.2,
 horizontal_flip = True)

train_generator = train_datagen.flow_from_directory(
    "/content/CovidDataset/Train",
    target_size = (224, 224),
    batch_size = 16,
    class_mode = 'binary')

validation_generator = val_dataset.flow_from_directory(
    "/content/CovidDataset/Val",
    target_size = (224, 224),
    batch_size = 16,
    class_mode = 'binary')

In [ ]:
train_generator.class_indices

{'Covid': 0, 'Normal': 1}

In [ ]:
validation_generator.class_indices

{'Covid': 0, 'Normal': 1}

## Model Creation

ORIGINAL model consist of : 

→ 2 x convolution layer of 64 channel of 3x3 kernal and same padding

→ 1 x maxpool layer of 2x2 pool size and stride 2x2

→ 2 x convolution layer of 128 channel of 3x3 kernal and same padding

→ 1 x maxpool layer of 2x2 pool size and stride 2x2

→ 3 x convolution layer of 256 channel of 3x3 kernal and same padding

→ 1 x maxpool layer of 2x2 pool size and stride 2x2

→ 3 x convolution layer of 512 channel of 3x3 kernal and same padding

→ 1 x maxpool layer of 2x2 pool size and stride 2x2

→ 3 x convolution layer of 512 channel of 3x3 kernal and same padding

→ 1 x maxpool layer of 2x2 pool size and stride 2x2

In [ ]:
# model = Sequential()
# model.add(Conv2D(input_shape=(224,224,3),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
# model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
# model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
# model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
# model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
# model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
# model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
# model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
# model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
# model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
# model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
# model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
# model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
# model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
# model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
# model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
# model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
# model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))


model = Sequential()
model.add(Conv2D(input_shape=(224,224,3),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

model.add(Flatten())
model.add(Dense(units=4096,activation="relu"))
model.add(Dense(units=4096,activation="relu"))
model.add(Dense(units=1, activation="sigmoid"))

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

## Training model and Testing

In [ ]:
hist = model.fit_generator(
    train_generator,
    steps_per_epoch=10,
    epochs = 15,
    validation_data = validation_generator,
    validation_steps=2,
  )

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/15
10/10 [==============================] - 14s 826ms/step - loss: 0.9760 - accuracy: 0.4959 - val_loss: 0.6894 - val_accuracy: 0.5625
Epoch 2/15
10/10 [==============================] - 7s 759ms/step - loss: 0.6961 - accuracy: 0.4475 - val_loss: 0.6924 - val_accuracy: 0.9062
Epoch 3/15
10/10 [==============================] - 8s 777ms/step - loss: 0.6878 - accuracy: 0.8136 - val_loss: 0.4572 - val_accuracy: 0.8125
Epoch 4/15
10/10 [==============================] - 7s 741ms/step - loss: 0.5766 - accuracy: 0.8341 - val_loss: 0.2070 - val_accuracy: 0.9062
Epoch 5/15
10/10 [==============================] - 8s 795ms/step - loss: 0.6651 - accuracy: 0.7213 - val_loss: 0.5701 - val_accuracy: 0.8750
Epoch 6/15
10/10 [==============================] - 7s 727ms/step - loss: 0.4859 - accuracy: 0.7920 - val_loss: 0.5639 - val_accuracy: 0.7188
Epoch 7/15
10/10 [==============================] - 8s 774ms/step - loss: 0.4406 - accuracy: 0.8141 - val_loss: 0.3981 - val_accuracy: 0.8125
Epoch

In [ ]:
model.evaluate(validation_generator)

14/14 [==============================] - 7s 498ms/step - loss: 0.2788 - accuracy: 0.8750


[0.2787880301475525, 0.875]